# Multistage Stochastic Unit Commitment Problem

In [66]:
import sys
sys.path.append('../sddip')

In [67]:
import os
import numpy as np
import pandas as pd
import gurobipy as gp
from scipy import stats, linalg

import sddip.config as config
import sddip.storage as storage
import sddip.utils as utils
import sddip.dualsolver as dualsolver
import sddip.ucmodel as ucmodel


## Parameters

In [68]:
test_case_raw_dir = "WB2/raw"

test_case_raw_dir = os.path.join(config.test_cases_dir, test_case_raw_dir)

bus_file_raw = os.path.join(test_case_raw_dir, "bus_data.txt")
branch_file_raw = os.path.join(test_case_raw_dir, "branch_data.txt")
gen_file_raw = os.path.join(test_case_raw_dir, "gen_data.txt")
gen_cost_file_raw = os.path.join(test_case_raw_dir, "gen_cost_data.txt")
scenario_data_file = os.path.join(test_case_raw_dir, "scenario_data.txt")

bus_df = pd.read_csv(bus_file_raw, delimiter="\t")
branch_df = pd.read_csv(branch_file_raw, delimiter="\t")
gen_df = pd.read_csv(gen_file_raw, delimiter="\t")
gen_cost_df = pd.read_csv(gen_cost_file_raw, delimiter="\t")

scenario_df = pd.read_csv(scenario_data_file, delimiter="\t")

In [69]:
nodes = bus_df.bus_i.values.tolist()
edges = branch_df[["fbus", "tbus"]].values.tolist()

graph = utils.Graph(nodes, edges)

ref_bus = bus_df.loc[bus_df.type == 3].bus_i.values[0]

a_inc = graph.incidence_matrix()
b_l = (-branch_df.x /(branch_df.r**2 + branch_df.x**2)).tolist()
b_diag = np.diag(b_l)

m1 = b_diag.dot(a_inc)
m2 = a_inc.T.dot(b_diag).dot(a_inc)

m1 = np.delete(m1, ref_bus-1, 1)
m2 = np.delete(m2, ref_bus-1, 0)
m2 = np.delete(m2, ref_bus-1, 1)

ptdf = m1.dot(np.linalg.inv(m2))

zeros_col = np.zeros((1,ptdf.shape[1]))

ptdf = np.insert(ptdf, ref_bus-1, zeros_col, axis=1)

ptdf

array([[ 0., -1.]])

In [70]:
########################################################################################################################
# Deterministic parameters
########################################################################################################################
gc = np.array(gen_cost_df.c1)
suc = np.array(gen_cost_df.startup)
sdc = np.array( gen_cost_df.startup)
pg_min = np.array(gen_df.Pmin)
pg_max = np.array(gen_df.Pmax)
pl_max = np.array(branch_df.rateA)

n_gens = len(gc)
n_lines, n_buses = ptdf.shape

# Lists of generators at each bus
#
# Example: [[0,1], [], [2]]
# Generator 1 & 2 are located at bus 1
# No Generator is located at bus 2
# Generator 3 is located at bus 3
gens_at_bus = [[] for _ in range(n_buses)]
g = 0
for b in gen_df.bus.values:
    gens_at_bus[b-1].append(g)
    g+=1

# TODO Add ramp rate limits
rg_up_max = np.full(n_gens, 1000)
rg_down_max = np.full(n_gens, 1000)

# TODO Adjust penalty
penalty = 10000
    
########################################################################################################################
# Stochastic parameters
########################################################################################################################
n_nodes_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()
n_stages = len(n_nodes_per_stage)

prob = []
p_d = []

for t in range(n_stages):
    stage_df = scenario_df[scenario_df["t"] == t+1]
    p_d.append(stage_df[scenario_df.columns[scenario_df.columns.to_series().str.contains('Pd')]].values.tolist())
    prob.append(stage_df["p"].values.tolist())


########################################################################################################################
# Expected values of stochastic parameters
########################################################################################################################
ex_pd = [np.array(prob[t]).dot(np.array(p_d[t])) for t in range(n_stages)]

In [71]:
# prob[t][n]
# Probability of realization n at stage t
prob

[[1.0], [0.5, 0.5]]

In [72]:
# p_d[t][n][b]
# Demand in stage t and realization n at bus b
p_d

[[[0, 350]], [[0, 300], [0, 200]]]

In [73]:
# ex_pd[t][b]
# Expected demand in stage t at bus b
ex_pd

[array([  0., 350.]), array([  0., 250.])]

## Optimization

### Variables

In [74]:
model = gp.Model("MSUC")

x = {}
y = {}
s_up = {}
s_down = {}
ys_p = {}
ys_n = {}

for t in range(n_stages):
    for n in range(n_nodes_per_stage[t]):
        for g in range(n_gens):
            x[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"x_{t+1}_{n+1}_{g+1}")
            y[t,n,g] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"y_{t+1}_{n+1}_{g+1}")
            s_up[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"s_up_{t+1}_{n+1}_{g+1}")
            s_down[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"s_down_{t+1}_{n+1}_{g+1}")
        ys_p[t,n] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_p_{t+1}_{n+1}")
        ys_n[t,n] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_n_{t+1}_{n+1}")

model.update()

### Constraints

In [75]:
# Objective
obj = gp.quicksum(prob[t][n]*(gc[g]*y[t,n,g] + suc[g]*s_up[t,n,g] + sdc[g]*s_down[t,n,g] + penalty*(ys_p[t,n] + ys_n[t,n])) 
                    for t in range(n_stages)
                    for n in range(n_nodes_per_stage[t])
                    for g in range(n_gens))

model.setObjective(obj)


# Balance constraints
model.addConstrs((gp.quicksum(y[t,n,g] for g in range(n_gens)) + ys_p[t,n] - ys_n[t,n] == gp.quicksum(p_d[t][n])
                    for t in range(n_stages)
                    for n in range(n_nodes_per_stage[t])),                    
                    "balance")


# Generator constraints
model.addConstrs((y[t,n,g] >= pg_min[g] 
                    for g in range(n_gens)
                    for t in range(n_stages) 
                    for n in range(n_nodes_per_stage[t])),
                    "min-generation")

model.addConstrs((y[t,n,g] <= pg_max[g] 
                    for g in range(n_gens)
                    for t in range(n_stages) 
                    for n in range(n_nodes_per_stage[t])),
                    "max-generation")


# Power flow constraints
for t in range(n_stages):
    for n in range(n_nodes_per_stage[t]):
        line_flows = [gp.quicksum(ptdf[l,b] * (gp.quicksum(y[t,n,g] for g in gens_at_bus[b]) - p_d[t][n][b])
                        for b in range(n_buses)) for l in range(n_lines)]
        model.addConstrs((line_flows[l] <= pl_max[l] for l in range(n_lines)), "power-flow(1)")
        model.addConstrs((-line_flows[l] <= pl_max[l] for l in range(n_lines)), "power-flow(2)")


# Startup shutdown constraints
# t=0
x_init = [0]*n_gens
model.addConstrs((x[0,0,g] - x_init[g] <= s_up[0,0,g] for g in range(n_gens)), "up-down(1)")
model.addConstrs((x[0,0,g] - x_init[g] <= s_up[0,0,g] - s_down[0,0,g] for g in range(n_gens)), "up-down(2)")
# t>0
for t in range(1,n_stages):
    for n in range(n_nodes_per_stage[t]):
        model.addConstrs((x[t,n,g] - x[t-1,n_prev,g] <= s_up[t,n,g] 
                        for g in range(n_gens) 
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "up-down(1)")
        model.addConstrs((x[t,n,g] - x[t-1,n_prev,g] <= s_up[t,n,g] - s_down[t,n,g] 
                        for g in range(n_gens)
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "up-down(2)")


# Ramp rate constraints
# t=0
y_init = [0]*n_gens
model.addConstrs((y[0,0,g] - y_init[g] <= rg_up_max[g] for g in range(n_gens)), "rate-up")
model.addConstrs((y_init[g] - y[0,0,g] <= rg_down_max[g] for g in range(n_gens)), "rate-down(2)")
# t>0
for t in range(1,n_stages):
    for n in range(n_nodes_per_stage[t]):
        model.addConstrs((y[t,n,g] - y[t-1,n_prev,g] <= rg_up_max[g]
                        for g in range(n_gens)
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "rate-up")
        model.addConstrs((y[t-1,n_prev,g] - y[t,n,g] <= rg_down_max[g]
                        for g in range(n_gens)
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "rate-down")

model.update()

### Solve

In [76]:
#model.setParam("OutputFlag",0)

model.optimize()

#model.setParam("OutputFlag",1)

model.printAttr("X")

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 27 rows, 18 columns and 44 nonzeros
Model fingerprint: 0x82ac71f4
Variable types: 9 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 1e+06]
Presolve removed 27 rows and 18 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 1200 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.200000000000e+03, best bound 1.200000000000e+03, gap 0.0000%

    Variable            X 
-------------------------
     y_1_1_1          350 
  s_up_1_1_1            1 
     y_2_1_1          300 
  s_up_2_1_1            1 
     y_2_2_1          200 
  s_up_2